In [1]:
import os
import json
import csv
import pandas as pd
import numpy as np
import math
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_v10.csv"
# results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_v11.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_CHAOS_v12.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_LVSC_v12.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_2_20_2024_merged_v13.csv"
results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_16_2023_merged_edited1.csv"

In [3]:
exp_df = pd.read_csv(results_csv_file)  
ignore_string_in_name_lst = ["strong", "sup"]
include_string_in_name_lst = ["v1"]

In [4]:
included_exp_names = exp_df['exp'].tolist()

In [5]:
def get_base_exp_name(exp_name):
    exp_field = "exp"
    exp_field_len_w_index = len(exp_field)+1
    exp_field_index = exp_name.find(exp_field)
    exp_name = exp_name[:exp_field_index]+exp_name[exp_field_index+exp_field_len_w_index:]
    return exp_name

In [6]:
exp_groups = dict()

for exp_name in included_exp_names:
    base_exp_name = get_base_exp_name(exp_name)
    if base_exp_name in exp_groups:
        exp_groups[base_exp_name] += [exp_name]
    else:
        exp_groups[base_exp_name] = [exp_name]

In [7]:
exp_length = 5
num_rounds = 10
round_strings =  ["round" + str(round_num) + "_test" for round_num in range(num_rounds)]
base_exp_names = list(exp_groups.keys())

In [8]:
results_dict = {}

for base_exp_name, exp_group in exp_groups.items():
    continue_flag = False
    for ignore_string_in_name in ignore_string_in_name_lst:
        if ignore_string_in_name in base_exp_name:
            continue_flag = True
            break
    for include_string_in_name in include_string_in_name_lst:
        if include_string_in_name not in base_exp_name:
            print("rejected, ", base_exp_name)
            continue_flag = True
            break
    if continue_flag:
        continue
    exp_dict = {}
    for i, run in enumerate(exp_group):
        sub_df = exp_df.loc[exp_df['exp'] == run]
        for round_string in round_strings:
            val = sub_df[round_string].iloc[0]
            if not math.isnan(val):
                if round_string not in exp_dict:
                    exp_dict[round_string] = [float(val)]
                else:
                    exp_dict[round_string] += [float(val)]
    results_dict[base_exp_name] = exp_dict
    print("accepted ", base_exp_name)

rejected,  DMPLS__coreset_cl_bs8_v5
rejected,  DMPLS__coreset_label2_uncertainty1_v8
rejected,  DMPLS__coreset_slice_pos10_v7
accepted  DMPLS__coreset_v10
rejected,  DMPLS__coreset_fuse10_extra_loss_wt06_use_slice_pos_v7
rejected,  DMPLS__coreset_fuse10_pos_loss1_wt02_pos_loss2_wt01_use_slice_pos_use_phase_bs9_v7
rejected,  DMPLS__coreset_label2_v7
rejected,  DMPLS_coregcn_bugfix
rejected,  DMPLS__coreset_cl_bs4_v5
rejected,  DMPLS__coreset_slice_pos10_patient10_v7
accepted  DMPLS__random_v10
accepted  DMPLS__vaal_v10
rejected,  DMPLS__coreset_cl_bs8_mdl_preds_v5
rejected,  DMPLS__coreset_preds_v5
rejected,  DMPLS__random_v7
rejected,  DMPLS__coreset_cl_bs8_mdl_ft_v6
rejected,  DMPLS__coreset_uncertainty1_v7
accepted  DMPLS__db_bald_v10
rejected,  DMPLS__coreset_v9
rejected,  DMPLS__coreset_cl_bs8_fuse05_v5
rejected,  DMPLS__coreset_cl_bs8_v5_
rejected,  DMPLS__coreset_slice_pos10_patient10_phase10_v7
rejected,  DMPLS__random_v8
accepted  DMPLS__random_v10_unused
rejected,  DMPLS__core

In [9]:
index = pd.Index(list(results_dict.keys()), name="exp_name")
round_counts_strings = [[round_string, f"round{i}_count"] for i, round_string in enumerate(round_strings)]
df = pd.DataFrame(index=index, columns=sum(round_counts_strings, []))

In [10]:
for base_exp_name, results in results_dict.items():
    sub_df = df.loc[base_exp_name]
    results = results_dict[base_exp_name]
    for round_string, count_string in round_counts_strings:
        if round_string in results:
            round_results = results[round_string]
            num_results = len(round_results)
            mean_result = np.mean(round_results)
            std_results = np.std(round_results, ddof=1)
            mean_string = '%.3f' % mean_result
            std_string = '%.3f'% std_results
            sub_df[round_string] = f"{mean_string}+/-{std_string}"
            sub_df[count_string] = num_results

/Users/arvin/.virtualenvs/al/lib/python3.9/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/arvin/.virtualenvs/al/lib/python3.9/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
df
df.sort_values(by=["round4_test"], ascending=False)

,round0_test,round0_count,round1_test,round1_count,round2_test,round2_count,round3_test,round3_count,round4_test,round4_count,round5_test,round5_count,round6_test,round6_count,round7_test,round7_count,round8_test,round8_count,round9_test,round9_count
exp_name,,,,,,,,,,,,,,,,,,,,
DMPLS__coreset_fuse20_v10,0.319+/-nan,1,0.644+/-nan,1,0.634+/-nan,1,0.715+/-nan,1,0.777+/-nan,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__coreset_fuse20_pos_loss1_wt02_pos_loss2_wt01_pos_loss3_wt005_use_slice_pos_use_phase_use_patient_uncertainty1_v10,0.333+/-0.072,5,0.556+/-0.070,5,0.614+/-0.042,5,0.737+/-0.034,5,0.775+/-0.019,5,0.801+/-0.005,5,0.838+/-0.009,5,0.843+/-0.006,5,0.864+/-0.005,5,NaN,NaN
DMPLS__coregcn_model_fts_ss10_rstart_v10,0.255+/-0.067,4,0.408+/-0.117,4,0.493+/-0.089,4,0.691+/-0.058,4,0.741+/-0.029,4,0.796+/-0.023,4,0.831+/-0.001,3,0.845+/-0.005,3,0.863+/-0.005,3,NaN,NaN
DMPLS__coreset_fuse10_pos_loss1_wt02_pos_loss2_wt01_pos_loss3_wt005_use_slice_pos_use_phase_use_patient_label2_uncertainty1_v10,0.318+/-0.101,5,0.471+/-0.112,5,0.594+/-0.058,5,0.696+/-0.027,5,0.741+/-0.021,5,0.803+/-0.011,5,0.839+/-0.009,5,0.842+/-0.008,5,0.863+/-0.012,5,NaN,NaN
DMPLS__random_v10_unused,0.420+/-nan,1,0.604+/-nan,1,0.498+/-nan,1,0.758+/-nan,1,0.724+/-nan,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__vaal_v10,NaN,NaN,0.459+/-0.116,5,0.478+/-0.082,4,0.661+/-0.101,4,0.721+/-0.070,4,0.799+/-0.008,4,0.823+/-0.014,4,0.838+/-0.013,4,0.864+/-0.005,4,NaN,NaN
DMPLS__coreset_v10,0.283+/-0.060,5,0.452+/-0.040,5,0.498+/-0.090,5,0.687+/-0.094,5,0.701+/-0.060,5,0.804+/-0.017,5,0.837+/-0.004,5,0.853+/-0.004,5,0.868+/-0.004,5,NaN,NaN
DMPLS__coreset_fuse20_uncertainty1_v10,0.256+/-0.038,4,0.480+/-0.103,4,0.515+/-0.085,4,0.614+/-0.086,4,0.692+/-0.089,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__coreset_fuse20_uncertainty1_label2_v10,0.358+/-0.019,3,0.489+/-0.041,3,0.486+/-0.076,3,0.617+/-0.077,3,0.687+/-0.052,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
